In [1]:
import os
from typing import List

import torch
from diffusers import StableDiffusionPipeline
from diffusers.pipelines.controlnet import MultiControlNetModel
from transformers import CLIPVisionModelWithProjection, CLIPImageProcessor
from PIL import Image

In [2]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipelineLegacy, DDIMScheduler, AutoencoderKL
from PIL import Image

In [3]:
base_model_path = "runwayml/stable-diffusion-v1-5"
vae_model_path = "stabilityai/sd-vae-ft-mse"

In [4]:

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path, cache_dir='/home/tyk/hf_cache').to(dtype=torch.float16)



pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None,
    cache_dir='/home/tyk/hf_cache'
)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [5]:
unet = pipe.unet

In [6]:
attn_procs = {}

In [12]:
# key와 value에 attention을 넣고 적용 # CA만 바꾸도록 
for name in unet.attn_processors.keys():
    # SA 제외하고
    cross_attention_dim = None if name.endswith("attn1.processor") else unet.config.cross_attention_dim
    # print(cross_attention_dim)

    if name.startswith("mid_block"):
        # 제일 안쪽
        hidden_size = unet.config.block_out_channels[-1]
        # print(hidden_size)
    elif name.startswith("up_blocks"):
        block_id = int(name[len("up_blocks.")])
        # print(name, block_id)
        hidden_size = list(reversed(unet.config.block_out_channels))[block_id]
        # print(hidden_size)
    elif name.startswith("down_blocks"):
        block_id = int(name[len("down_blocks.")])
        hidden_size = unet.config.block_out_channels[block_id]
    # SA이라면
    if cross_attention_dim is None:
        attn_procs[name] = AttnProcessor()
    else:
        attn_procs[name] = IPAttnProcessor(hidden_size=hidden_size, cross_attention_dim=cross_attention_dim,
                scale=1.0,num_tokens= self.num_tokens).to(self.device, dtype=torch.float16)

unet.set_attn_processor(attn_procs)
if hasattr(self.pipe, "controlnet"):
    if isinstance(self.pipe.controlnet, MultiControlNetModel):
        for controlnet in self.pipe.controlnet.nets:
            controlnet.set_attn_processor(CNAttnProcessor(num_tokens=self.num_tokens))
    else:
        self.pipe.controlnet.set_attn_processor(CNAttnProcessor(num_tokens=self.num_tokens))

up_blocks.1.attentions.0.transformer_blocks.0.attn1.processor 1
1280
up_blocks.1.attentions.0.transformer_blocks.0.attn2.processor 1
1280
up_blocks.1.attentions.1.transformer_blocks.0.attn1.processor 1
1280
up_blocks.1.attentions.1.transformer_blocks.0.attn2.processor 1
1280
up_blocks.1.attentions.2.transformer_blocks.0.attn1.processor 1
1280
up_blocks.1.attentions.2.transformer_blocks.0.attn2.processor 1
1280
up_blocks.2.attentions.0.transformer_blocks.0.attn1.processor 2
640
up_blocks.2.attentions.0.transformer_blocks.0.attn2.processor 2
640
up_blocks.2.attentions.1.transformer_blocks.0.attn1.processor 2
640
up_blocks.2.attentions.1.transformer_blocks.0.attn2.processor 2
640
up_blocks.2.attentions.2.transformer_blocks.0.attn1.processor 2
640
up_blocks.2.attentions.2.transformer_blocks.0.attn2.processor 2
640
up_blocks.3.attentions.0.transformer_blocks.0.attn1.processor 3
320
up_blocks.3.attentions.0.transformer_blocks.0.attn2.processor 3
320
up_blocks.3.attentions.1.transformer_block